In [1]:
import os
import shutil
import random

def partition_images(base_dir, train_dir, test_dir, train_ratio):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    all_images = os.listdir(base_dir)
    random.shuffle(all_images)

    train_count = int(len(all_images) * train_ratio)
    train_images = all_images[:train_count]
    test_images = all_images[train_count:]

    for image in train_images:
        shutil.copyfile(os.path.join(base_dir, image), os.path.join(train_dir, image))

    for image in test_images:
        shutil.copyfile(os.path.join(base_dir, image), os.path.join(test_dir, image))

partition_images('images', 'train', 'test', 0.8)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 8
learning_rate = 0.001
num_epochs = 300

model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

image_dir = "train"
dataset = CustomDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

total_steps = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

        # if (i + 1) % 10 == 0:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "cnn_model_500images.pth")


Epoch [1/500], Step [1/26], Loss: 50644.3555
Epoch [1/500], Step [2/26], Loss: 9929.5088
Epoch [1/500], Step [3/26], Loss: 35850.0938
Epoch [1/500], Step [4/26], Loss: 11513.4434
Epoch [1/500], Step [5/26], Loss: 7076.7783
Epoch [1/500], Step [6/26], Loss: 8970.6182
Epoch [1/500], Step [7/26], Loss: 10255.7510
Epoch [1/500], Step [8/26], Loss: 12498.0508
Epoch [1/500], Step [9/26], Loss: 5950.2588
Epoch [1/500], Step [10/26], Loss: 10170.3750
Epoch [1/500], Step [11/26], Loss: 6176.5229
Epoch [1/500], Step [12/26], Loss: 9368.8809
Epoch [1/500], Step [13/26], Loss: 7273.3184
Epoch [1/500], Step [14/26], Loss: 5982.3408
Epoch [1/500], Step [15/26], Loss: 5375.0225
Epoch [1/500], Step [16/26], Loss: 8632.0098
Epoch [1/500], Step [17/26], Loss: 8417.8438
Epoch [1/500], Step [18/26], Loss: 9049.4033
Epoch [1/500], Step [19/26], Loss: 14023.5908
Epoch [1/500], Step [20/26], Loss: 10278.7471
Epoch [1/500], Step [21/26], Loss: 8028.3350
Epoch [1/500], Step [22/26], Loss: 8640.2666
Epoch [1/50

KeyboardInterrupt: 

In [5]:
import torch
import os
from torch.utils.data import DataLoader
import math
import numpy as np

def evaluate_model(model, test_dir):
    model.eval()
    test_dataset = CustomDataset(test_dir)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(2, device=device)   # Added device argument
    total_mape = torch.zeros(2, device=device)  # Added device argument
    total_smape = torch.zeros(2, device=device) # Added device argument
    total_mse = torch.zeros(2, device=device)   # Added device argument
    total_count = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)  # Absolute error
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8  # Only calculate percentage for non-zero labels
            percentage_error = (absolute_error / torch.abs(labels)) * 100  # Percentage error
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2  # Mean Square Error
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)  # number of samples

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()  # Moved tensors to CPU and converted to numpy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN()
model.load_state_dict(torch.load("cnn_model_500images.pth"))
model.to(device)

mae, mape, smape, rmse = evaluate_model(model, "test")
for i in range(2):
    print(f"Metrics for prediction {i+1}:")
    print(f"Mean Absolute Error: {mae[i]:.4f}")
    print(f"Mean Absolute Percentage Error: {mape[i]:.2f}%")
    print(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%")
    print(f"Root Mean Square Error: {rmse[i]:.4f}")
    print()


Metrics for prediction 1:
Mean Absolute Error: 0.7092
Mean Absolute Percentage Error: 51.98%
Symmetric Mean Absolute Percentage Error: 41.23%
Root Mean Square Error: 0.8632

Metrics for prediction 2:
Mean Absolute Error: 42.4921
Mean Absolute Percentage Error: 19.54%
Symmetric Mean Absolute Percentage Error: 17.13%
Root Mean Square Error: 60.1152



In [ ]:
import torchvision.transforms as transforms
import cv2

model = CNN()
model.load_state_dict(torch.load("cnn_model.pth"))
model.eval()

# using cv2 is gross but i like the transforms more
image_path = "test\epsnx0.10_alfax-0.00_betax282.89_epsny0.10_alfay-0.55_betay170.00_epsnz5.00_alfaz0.10_betaz10.00.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = transforms.ToTensor()(image)
image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
image = image.unsqueeze(0)

with torch.no_grad():
    output = model(image)
    predicted_variables = output.squeeze().tolist()

print("Predicted variables:", predicted_variables)

Predicted variables: [0.011563804000616074, 281.4032897949219]
